In [1]:
import re
import spacy
import gensim
import logging
import pandas as pd
import numpy as np
# remove warnings
logging.basicConfig(level=logging.ERROR)
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# lectura de la data y eliminación de registros repetidos y nulos
documents = pd.read_csv('./datos/simpsons_dataset.csv').dropna().drop_duplicates()
documents = documents.reset_index(drop=True)
print(documents.shape)
display(documents.head())

(126646, 2)


,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [3]:
# Filtrado por personajes principales
main_characters =  ["Lisa Simpson", "Bart Simpson", "Homer Simpson", "Marge Simpson"]
documents = documents[documents["raw_character_text"].isin(main_characters)]
print(documents.shape)
display(documents.head())

(60610, 2)


,raw_character_text,spoken_words
1,Lisa Simpson,Where's Mr. Bergstrom?
3,Lisa Simpson,That life is worth living.
7,Bart Simpson,Victory party under the slide!
8,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!
10,Lisa Simpson,Do you know where I could find him?


In [4]:
def get_sublist(list:list, n:int) -> list:
    """
    Obtener sublistas de una lista
    """
    return [list[i:i+n] for i in range(0, len(list), n)]


In [5]:
def get_vectors_characters(characters: pd.DataFrame) -> pd.DataFrame:
    """
    Extraer dialogos para todos los personajes en grupos de 5 oraciones
    """
    sentences_big = get_sublist(characters['spoken_words'].tolist(),5)
    df = pd.DataFrame({ 'spoken_words': sentences_big})
    for index,dialogue in df['spoken_words'].items():
        df.loc[index,"spoken_words_concatenated"] = ' '.join(dialogue)
    return df

In [6]:
documents_dialogue_concatenated = documents.groupby("raw_character_text")  \
                                    .apply(lambda x: get_vectors_characters(x)) \
                                    .reset_index("raw_character_text")

In [7]:
def convert_label(label:str) -> int:
    """ 
    Categoriza los labels
    """
    if label == "bart simpson":
        return 0
    elif label == "lisa simpson":
        return 1
    elif label == "homer simpson":
        return 2
    else : 
        return 3
   

In [8]:
documents_dialogue_concatenated["raw_character_text"] = documents_dialogue_concatenated["raw_character_text"].apply(lambda x: x.lower())
documents_dialogue_concatenated["raw_character_text"] = documents_dialogue_concatenated["raw_character_text"].apply(convert_label)
documents_dialogue_concatenated["spoken_words_concatenated"] = documents_dialogue_concatenated["spoken_words_concatenated"].apply(lambda x: x.lower())
documents_dialogue_concatenated.head()

,raw_character_text,spoken_words,spoken_words_concatenated
0,1,"[Victory party under the slide!, Hey, thanks f...","victory party under the slide! hey, thanks for..."
1,1,"[Somebody must have voted., Uh oh., I demand a...",somebody must have voted. uh oh. i demand a re...
2,1,"[Ah, Dad, if just me, Milhouse and Lewis had v...","ah, dad, if just me, milhouse and lewis had vo..."
3,1,"[Please Dad., What?, Yes sir., They're fightin...",please dad. what? yes sir. they're fighting in...
4,1,"[Dad, I have as much respect for you as I ever...","dad, i have as much respect for you as i ever ..."


In [9]:

sentences = documents_dialogue_concatenated['spoken_words_concatenated'].values
y = documents_dialogue_concatenated['raw_character_text'].values
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.2, random_state=1)

y_train_categorical = tf.one_hot(y_train,4)
y_test_categorical = tf.one_hot(y_test,4)


train_dataset = tf.data.Dataset.from_tensor_slices((sentences_train,y_train_categorical))
test_dataset = tf.data.Dataset.from_tensor_slices((sentences_test,y_test_categorical))

text_dataset = tf.data.Dataset.from_tensor_slices((sentences_train))

In [10]:

print('Number of rows in the total set: {}'.format(sentences.shape[0]))
print('Number of rows of bart class: {}'.format(y[y == 0].shape[0]))
print('Number of rows of lisa class: {}'.format(y[y == 1].shape[0]))
print('Number of rows of homer class: {}'.format(y[y == 2].shape[0]))
print('Number of rows of marge class: {}'.format(y[y == 3].shape[0]))
print('---------------------------------------------')

print('Number of rows in the training set: {}'.format(sentences_train.shape[0]))
print('Number of rows of bart class: {}'.format(y_train[y_train == 0].shape[0]))
print('Number of rows of lisa class: {}'.format(y_train[y_train == 1].shape[0]))
print('Number of rows of homer class: {}'.format(y_train[y_train == 2].shape[0]))
print('Number of rows of marge class: {}'.format(y_train[y_train == 3].shape[0]))
print('---------------------------------------------')

print('Number of rows in the test set: {}'.format(sentences_test.shape[0]))

print('Number of rows of bart class: {}'.format(y_test[y_test == 0].shape[0]))
print('Number of rows of lisa class: {}'.format(y_test[y_test == 1].shape[0]))
print('Number of rows of homer class: {}'.format(y_test[y_test == 2].shape[0]))
print('Number of rows of marge class: {}'.format(y_test[y_test == 3].shape[0]))


Number of rows in the total set: 12123
Number of rows of bart class: 2414
Number of rows of lisa class: 2029
Number of rows of homer class: 5195
Number of rows of marge class: 2485
---------------------------------------------
Number of rows in the training set: 9698
Number of rows of bart class: 1913
Number of rows of lisa class: 1598
Number of rows of homer class: 4171
Number of rows of marge class: 2016
---------------------------------------------
Number of rows in the test set: 2425
Number of rows of bart class: 501
Number of rows of lisa class: 431
Number of rows of homer class: 1024
Number of rows of marge class: 469


# Arquitectura 1

## Vectorize Layer Binary

In [11]:
vectorize_layer_binary = TextVectorization(
    ngrams=None, 
    max_tokens=200, vocabulary=None,
    output_mode='binary', output_sequence_length=None, pad_to_max_tokens=True,
)

vectorize_layer_binary.adapt(text_dataset)


In [12]:
vocab_size = len(vectorize_layer_binary.get_vocabulary())
vocab_size

200

In [13]:
embedding_dim = 100

In [14]:
modelsequ1 = Sequential()
modelsequ1.add(vectorize_layer_binary)
modelsequ1.add(layers.Embedding(vocab_size, embedding_dim, name="embedding"))
modelsequ1.add(layers.GlobalAveragePooling1D())
modelsequ1.add(layers.Dense(50, activation='relu', name="Hidden_layer"))
modelsequ1.add(layers.Dense(4, activation='softmax', name="Output_layer"))
modelsequ1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', 'mse']) 


In [15]:
batch_size = 100
epochs = 10

# Fit the model using the train and test datasets.
modelsequ1.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/10
97/97 [==============================] - 4s 23ms/step - loss: 0.9290 - acc: 0.4227 - mse: 0.1447 - val_loss: 0.8613 - val_acc: 0.4223 - val_mse: 0.1430
Epoch 2/10
97/97 [==============================] - 2s 19ms/step - loss: 0.8075 - acc: 0.4301 - mse: 0.1402 - val_loss: 0.8279 - val_acc: 0.4223 - val_mse: 0.1427
Epoch 3/10
97/97 [==============================] - 2s 19ms/step - loss: 0.8047 - acc: 0.4301 - mse: 0.1405 - val_loss: 0.8287 - val_acc: 0.4223 - val_mse: 0.1436
Epoch 4/10
97/97 [==============================] - 2s 19ms/step - loss: 0.8088 - acc: 0.4291 - mse: 0.1413 - val_loss: 0.8642 - val_acc: 0.4223 - val_mse: 0.1505
Epoch 5/10
97/97 [==============================] - 2s 19ms/step - loss: 0.8258 - acc: 0.4065 - mse: 0.1450 - val_loss: 0.8912 - val_acc: 0.2066 - val_mse: 0.1533
Epoch 6/10
97/97 [==============================] - 2s 20ms/step - loss: 0.8316 - acc: 0.3946 - mse: 0.1457 - val_loss: 0.9167 - val_acc: 0.2066 - val_mse: 0.1558
Epoch 7/10
97/97 [====

In [16]:
modelsequ1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 200)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 200, 100)          20000     
                                                                 
 global_average_pooling1d (G  (None, 100)              0         
 lobalAveragePooling1D)                                          
                                                                 
 Hidden_layer (Dense)        (None, 50)                5050      
                                                                 
 Output_layer (Dense)        (None, 4)                 204       
                                                                 
Total params: 25,254
Trainable params: 25,254
Non-traina

In [17]:
modelsequ1.evaluate(sentences_test,y_test_categorical)

76/76 [==============================] - 0s 5ms/step - loss: 0.9053 - acc: 0.4223 - mse: 0.1528


[0.9053424000740051, 0.4222680330276489, 0.15278129279613495]

In [18]:
pred = modelsequ1.predict(sentences_test, batch_size=100, verbose=1) 
predicted = np.argmax(pred, axis=1)  
report = classification_report(np.argmax(y_test_categorical, axis=1), predicted)  
print(report)

25/25 [==============================] - 1s 10ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       469
           1       0.00      0.00      0.00       501
           2       0.00      0.00      0.00       431
           3       0.42      1.00      0.59      1024

    accuracy                           0.42      2425
   macro avg       0.11      0.25      0.15      2425
weighted avg       0.18      0.42      0.25      2425



c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Vectorize Layer count

In [19]:
vectorize_layer_count = TextVectorization(
    ngrams=None, 
    max_tokens=200, vocabulary=None,
    output_mode='count', output_sequence_length=None, pad_to_max_tokens=True,
)

vectorize_layer_count.adapt(text_dataset)

In [20]:
embedding_dim = 100
vocab_size = len(vectorize_layer_count.get_vocabulary())
vocab_size

200

In [21]:
modelsequ2 = Sequential()
modelsequ2.add(vectorize_layer_count)
modelsequ2.add(layers.Embedding(vocab_size, embedding_dim, name="embedding"))
modelsequ2.add(layers.GlobalAveragePooling1D())
modelsequ2.add(layers.Dense(50, activation='relu', name="Hidden_layer"))
modelsequ2.add(layers.Dense(4, activation='softmax', name="Output_layer"))
modelsequ2.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc', 'mse']) 


In [22]:
# Fit the model using the train and test datasets.
batch_size = 100
epochs = 10

modelsequ2.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/10
97/97 [==============================] - 4s 28ms/step - loss: 1.0375 - acc: 0.4066 - mse: 0.1541 - val_loss: 1.0105 - val_acc: 0.4223 - val_mse: 0.1503
Epoch 2/10
97/97 [==============================] - 3s 27ms/step - loss: 0.9539 - acc: 0.4301 - mse: 0.1445 - val_loss: 0.9515 - val_acc: 0.4223 - val_mse: 0.1444
Epoch 3/10
97/97 [==============================] - 3s 26ms/step - loss: 0.9077 - acc: 0.4301 - mse: 0.1404 - val_loss: 0.9190 - val_acc: 0.4223 - val_mse: 0.1422
Epoch 4/10
97/97 [==============================] - 2s 25ms/step - loss: 0.8811 - acc: 0.4301 - mse: 0.1389 - val_loss: 0.8990 - val_acc: 0.4223 - val_mse: 0.1415
Epoch 5/10
97/97 [==============================] - 2s 24ms/step - loss: 0.8643 - acc: 0.4301 - mse: 0.1383 - val_loss: 0.8854 - val_acc: 0.4223 - val_mse: 0.1412
Epoch 6/10
97/97 [==============================] - 2s 22ms/step - loss: 0.8526 - acc: 0.4301 - mse: 0.1381 - val_loss: 0.8753 - val_acc: 0.4223 - val_mse: 0.1411
Epoch 7/10
97/97 [====

In [23]:
modelsequ2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 200)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 200, 100)          20000     
                                                                 
 global_average_pooling1d_1   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 Hidden_layer (Dense)        (None, 50)                5050      
                                                                 
 Output_layer (Dense)        (None, 4)                 204       
                                                                 
Total params: 25,254
Trainable params: 25,254
Non-trai

In [24]:
modelsequ2.evaluate(sentences_test,y_test_categorical)

76/76 [==============================] - 0s 6ms/step - loss: 0.8523 - acc: 0.4223 - mse: 0.1410


[0.8523479700088501, 0.4222680330276489, 0.14097552001476288]

In [25]:
pred = modelsequ2.predict(sentences_test, batch_size=100, verbose=1) 
predicted = np.argmax(pred, axis=1)  
report = classification_report(np.argmax(y_test_categorical, axis=1), predicted)  
print(report)

25/25 [==============================] - 0s 10ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       469
           1       0.00      0.00      0.00       501
           2       0.00      0.00      0.00       431
           3       0.42      1.00      0.59      1024

    accuracy                           0.42      2425
   macro avg       0.11      0.25      0.15      2425
weighted avg       0.18      0.42      0.25      2425



c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Vectorize layer int

In [26]:
vectorize_layer_int = TextVectorization(
    ngrams=None, 
    max_tokens=200, vocabulary=None,
    output_mode='int', output_sequence_length=None,
)

vectorize_layer_int.adapt(text_dataset)

In [27]:
embedding_dim = 100
vocab_size = len(vectorize_layer_int.get_vocabulary())
vocab_size

200

In [28]:
modelsequ3 = Sequential()
modelsequ3.add(vectorize_layer_int)
modelsequ3.add(layers.Embedding(vocab_size, embedding_dim, name="embedding"))
modelsequ3.add(layers.GlobalAveragePooling1D())
modelsequ3.add(layers.Dense(50, activation='relu', name="Hidden_layer"))
modelsequ3.add(layers.Dense(4, activation='softmax', name="Output_layer"))
modelsequ3.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc', 'mse']) 

In [29]:
# Fit the model using the train and test datasets.
modelsequ3.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/10
97/97 [==============================] - 3s 19ms/step - loss: 1.0193 - acc: 0.4289 - mse: 0.1520 - val_loss: 0.9894 - val_acc: 0.4223 - val_mse: 0.1482
Epoch 2/10
97/97 [==============================] - 2s 17ms/step - loss: 0.9341 - acc: 0.4301 - mse: 0.1429 - val_loss: 0.9352 - val_acc: 0.4223 - val_mse: 0.1434
Epoch 3/10
97/97 [==============================] - 2s 18ms/step - loss: 0.8932 - acc: 0.4301 - mse: 0.1398 - val_loss: 0.9073 - val_acc: 0.4223 - val_mse: 0.1419
Epoch 4/10
97/97 [==============================] - 2s 17ms/step - loss: 0.8710 - acc: 0.4301 - mse: 0.1388 - val_loss: 0.8910 - val_acc: 0.4223 - val_mse: 0.1415
Epoch 5/10
97/97 [==============================] - 2s 17ms/step - loss: 0.8575 - acc: 0.4301 - mse: 0.1385 - val_loss: 0.8800 - val_acc: 0.4223 - val_mse: 0.1414
Epoch 6/10
97/97 [==============================] - 2s 18ms/step - loss: 0.8481 - acc: 0.4301 - mse: 0.1384 - val_loss: 0.8721 - val_acc: 0.4223 - val_mse: 0.1413
Epoch 7/10
97/97 [====

In [30]:

modelsequ3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_2 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, None, 100)         20000     
                                                                 
 global_average_pooling1d_2   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 Hidden_layer (Dense)        (None, 50)                5050      
                                                                 
 Output_layer (Dense)        (None, 4)                 204       
                                                                 
Total params: 25,254
Trainable params: 25,254
Non-trai

In [31]:
modelsequ3.evaluate(sentences_test,y_test_categorical)

76/76 [==============================] - 0s 4ms/step - loss: 0.8587 - acc: 0.4223 - mse: 0.1412


[0.8586507439613342, 0.4222680330276489, 0.14121271669864655]

In [32]:
pred = modelsequ3.predict(sentences_test, batch_size=100, verbose=1) 
predicted = np.argmax(pred, axis=1)  
report = classification_report(np.argmax(y_test_categorical, axis=1), predicted)  
print(report)

25/25 [==============================] - 0s 7ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       469
           1       0.00      0.00      0.00       501
           2       0.00      0.00      0.00       431
           3       0.42      1.00      0.59      1024

    accuracy                           0.42      2425
   macro avg       0.11      0.25      0.15      2425
weighted avg       0.18      0.42      0.25      2425



c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Arquitectura 2

## Vectorize Layer Binary

In [33]:
vectorize_layer_binary = TextVectorization(
    ngrams=None, 
    max_tokens=200, vocabulary=None,
    output_mode='binary', output_sequence_length=None, pad_to_max_tokens=True,
)

vectorize_layer_binary.adapt(text_dataset)


In [34]:
vocab_size = len(vectorize_layer_binary.get_vocabulary())
vocab_size

200

In [35]:
embedding_dim = 100

In [36]:
modelsequ1 = Sequential()
modelsequ1.add(vectorize_layer_binary)
modelsequ1.add(layers.Embedding(vocab_size, embedding_dim, name="embedding"))
modelsequ1.add(layers.GlobalAveragePooling1D())
modelsequ1.add(layers.Dense(50, activation='relu', name="Hidden_layer"))
modelsequ1.add(layers.Dense(50, activation='relu', name="Hidden_layer_2"))
modelsequ1.add(layers.Dense(4, activation='softmax', name="Output_layer"))
modelsequ1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', 'mse']) 


In [37]:
batch_size = 100
epochs = 10

# Fit the model using the train and test datasets.
modelsequ1.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/10
97/97 [==============================] - 4s 24ms/step - loss: 0.8940 - acc: 0.4295 - mse: 0.1438 - val_loss: 0.8269 - val_acc: 0.4223 - val_mse: 0.1432
Epoch 2/10
97/97 [==============================] - 2s 21ms/step - loss: 0.8058 - acc: 0.4301 - mse: 0.1414 - val_loss: 0.8625 - val_acc: 0.4223 - val_mse: 0.1482
Epoch 3/10
97/97 [==============================] - 2s 20ms/step - loss: 0.8670 - acc: 0.3788 - mse: 0.1521 - val_loss: 0.9076 - val_acc: 0.4223 - val_mse: 0.1531
Epoch 4/10
97/97 [==============================] - 2s 20ms/step - loss: 1.4629 - acc: 0.3180 - mse: 0.2084 - val_loss: 1.4051 - val_acc: 0.2066 - val_mse: 0.2061
Epoch 5/10
97/97 [==============================] - 2s 20ms/step - loss: 2.9974 - acc: 0.3023 - mse: 0.2530 - val_loss: 3.2271 - val_acc: 0.4223 - val_mse: 0.2343
Epoch 6/10
97/97 [==============================] - 2s 20ms/step - loss: 13.0034 - acc: 0.3121 - mse: 0.2821 - val_loss: 15.3277 - val_acc: 0.2066 - val_mse: 0.3484
Epoch 7/10
97/97 [==

In [38]:
modelsequ1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_3 (TextV  (None, 200)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 200, 100)          20000     
                                                                 
 global_average_pooling1d_3   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 Hidden_layer (Dense)        (None, 50)                5050      
                                                                 
 Hidden_layer_2 (Dense)      (None, 50)                2550      
                                                                 
 Output_layer (Dense)        (None, 4)                

In [39]:
modelsequ1.evaluate(sentences_test,y_test_categorical)

76/76 [==============================] - 1s 5ms/step - loss: 46.1362 - acc: 0.4223 - mse: 0.2405


[46.13624572753906, 0.4222680330276489, 0.24051547050476074]

In [40]:
pred = modelsequ1.predict(sentences_test, batch_size=100, verbose=1) 
predicted = np.argmax(pred, axis=1)  
report = classification_report(np.argmax(y_test_categorical, axis=1), predicted)  
print(report)

25/25 [==============================] - 0s 9ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       469
           1       0.00      0.00      0.00       501
           2       0.00      0.00      0.00       431
           3       0.42      1.00      0.59      1024

    accuracy                           0.42      2425
   macro avg       0.11      0.25      0.15      2425
weighted avg       0.18      0.42      0.25      2425



c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Vectorize Layer count

In [41]:
vectorize_layer_count = TextVectorization(
    ngrams=None, 
    max_tokens=200, vocabulary=None,
    output_mode='count', output_sequence_length=None, pad_to_max_tokens=True,
)

vectorize_layer_count.adapt(text_dataset)

In [42]:
embedding_dim = 100
vocab_size = len(vectorize_layer_count.get_vocabulary())
vocab_size

200

In [43]:
modelsequ2 = Sequential()
modelsequ2.add(vectorize_layer_count)
modelsequ2.add(layers.Embedding(vocab_size, embedding_dim, name="embedding"))
modelsequ2.add(layers.GlobalAveragePooling1D())
modelsequ2.add(layers.Dense(50, activation='relu', name="Hidden_layer"))
modelsequ2.add(layers.Dense(50, activation='relu', name="Hidden_layer_2"))
modelsequ2.add(layers.Dense(4, activation='softmax', name="Output_layer"))
modelsequ2.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc', 'mse']) 


In [44]:
# Fit the model using the train and test datasets.
batch_size = 100
epochs = 10

modelsequ2.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/10
97/97 [==============================] - 4s 27ms/step - loss: 1.0164 - acc: 0.4280 - mse: 0.1515 - val_loss: 0.9785 - val_acc: 0.4223 - val_mse: 0.1466
Epoch 2/10
97/97 [==============================] - 2s 24ms/step - loss: 0.9212 - acc: 0.4301 - mse: 0.1412 - val_loss: 0.9264 - val_acc: 0.4223 - val_mse: 0.1427
Epoch 3/10
97/97 [==============================] - 2s 24ms/step - loss: 0.8847 - acc: 0.4301 - mse: 0.1392 - val_loss: 0.9045 - val_acc: 0.4223 - val_mse: 0.1422
Epoch 4/10
97/97 [==============================] - 3s 30ms/step - loss: 0.8670 - acc: 0.4301 - mse: 0.1388 - val_loss: 0.8901 - val_acc: 0.4223 - val_mse: 0.1420
Epoch 5/10
97/97 [==============================] - 3s 32ms/step - loss: 0.8536 - acc: 0.4301 - mse: 0.1386 - val_loss: 0.8767 - val_acc: 0.4223 - val_mse: 0.1417
Epoch 6/10
97/97 [==============================] - 3s 35ms/step - loss: 0.8418 - acc: 0.4301 - mse: 0.1384 - val_loss: 0.8648 - val_acc: 0.4223 - val_mse: 0.1414
Epoch 7/10
97/97 [====

In [45]:
modelsequ2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_4 (TextV  (None, 200)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 200, 100)          20000     
                                                                 
 global_average_pooling1d_4   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 Hidden_layer (Dense)        (None, 50)                5050      
                                                                 
 Hidden_layer_2 (Dense)      (None, 50)                2550      
                                                                 
 Output_layer (Dense)        (None, 4)                

In [46]:
modelsequ2.evaluate(sentences_test,y_test_categorical)

76/76 [==============================] - 1s 6ms/step - loss: 1.0368 - acc: 0.4223 - mse: 0.1544


[1.0368092060089111, 0.4222680330276489, 0.1543927937746048]

In [47]:
pred = modelsequ2.predict(sentences_test, batch_size=100, verbose=1) 
predicted = np.argmax(pred, axis=1)  
report = classification_report(np.argmax(y_test_categorical, axis=1), predicted)  
print(report)

25/25 [==============================] - 1s 11ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       469
           1       0.00      0.00      0.00       501
           2       0.00      0.00      0.00       431
           3       0.42      1.00      0.59      1024

    accuracy                           0.42      2425
   macro avg       0.11      0.25      0.15      2425
weighted avg       0.18      0.42      0.25      2425



c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Vectorize layer int

In [48]:
vectorize_layer_int = TextVectorization(
    ngrams=None, 
    max_tokens=200, vocabulary=None,
    output_mode='int', output_sequence_length=None,
)

vectorize_layer_int.adapt(text_dataset)

In [49]:
embedding_dim = 100
vocab_size = len(vectorize_layer_int.get_vocabulary())
vocab_size

200

In [50]:
modelsequ3 = Sequential()
modelsequ3.add(vectorize_layer_int)
modelsequ3.add(layers.Embedding(vocab_size, embedding_dim, name="embedding"))
modelsequ3.add(layers.GlobalAveragePooling1D())
modelsequ3.add(layers.Dense(50, activation='relu', name="Hidden_layer"))
modelsequ3.add(layers.Dense(50, activation='relu', name="Hidden_layer_2"))
modelsequ3.add(layers.Dense(4, activation='softmax', name="Output_layer"))
modelsequ3.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc', 'mse']) 

In [51]:
# Fit the model using the train and test datasets.
modelsequ3.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/10
97/97 [==============================] - 3s 24ms/step - loss: 1.0426 - acc: 0.4172 - mse: 0.1548 - val_loss: 1.0164 - val_acc: 0.4223 - val_mse: 0.1512
Epoch 2/10
97/97 [==============================] - 3s 27ms/step - loss: 0.9608 - acc: 0.4301 - mse: 0.1455 - val_loss: 0.9562 - val_acc: 0.4223 - val_mse: 0.1451
Epoch 3/10
97/97 [==============================] - 3s 30ms/step - loss: 0.9114 - acc: 0.4301 - mse: 0.1410 - val_loss: 0.9189 - val_acc: 0.4223 - val_mse: 0.1423
Epoch 4/10
97/97 [==============================] - 2s 23ms/step - loss: 0.8801 - acc: 0.4301 - mse: 0.1390 - val_loss: 0.8946 - val_acc: 0.4223 - val_mse: 0.1412
Epoch 5/10
97/97 [==============================] - 3s 27ms/step - loss: 0.8594 - acc: 0.4301 - mse: 0.1382 - val_loss: 0.8778 - val_acc: 0.4223 - val_mse: 0.1408
Epoch 6/10
97/97 [==============================] - 3s 26ms/step - loss: 0.8447 - acc: 0.4301 - mse: 0.1379 - val_loss: 0.8652 - val_acc: 0.4223 - val_mse: 0.1407
Epoch 7/10
97/97 [====

In [52]:

modelsequ3.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_5 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, None, 100)         20000     
                                                                 
 global_average_pooling1d_5   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 Hidden_layer (Dense)        (None, 50)                5050      
                                                                 
 Hidden_layer_2 (Dense)      (None, 50)                2550      
                                                                 
 Output_layer (Dense)        (None, 4)                

In [53]:
modelsequ3.evaluate(sentences_test,y_test_categorical)

76/76 [==============================] - 0s 4ms/step - loss: 0.8509 - acc: 0.4223 - mse: 0.1404


[0.8509414196014404, 0.4222680330276489, 0.14042586088180542]

In [54]:
pred = modelsequ3.predict(sentences_test, batch_size=100, verbose=1) 
predicted = np.argmax(pred, axis=1)  
report = classification_report(np.argmax(y_test_categorical, axis=1), predicted)  
print(report)

25/25 [==============================] - 0s 7ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       469
           1       0.00      0.00      0.00       501
           2       0.00      0.00      0.00       431
           3       0.42      1.00      0.59      1024

    accuracy                           0.42      2425
   macro avg       0.11      0.25      0.15      2425
weighted avg       0.18      0.42      0.25      2425



c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Arquitectura 3

## Vectorize Layer Binary

In [55]:
vectorize_layer_binary = TextVectorization(
    ngrams=None, 
    max_tokens=200, vocabulary=None,
    output_mode='binary', output_sequence_length=None, pad_to_max_tokens=True,
)

vectorize_layer_binary.adapt(text_dataset)


In [56]:
vocab_size = len(vectorize_layer_binary.get_vocabulary())
vocab_size

200

In [57]:
embedding_dim = 100

In [58]:
modelsequ1 = Sequential()
modelsequ1.add(vectorize_layer_binary)
modelsequ1.add(layers.Embedding(vocab_size, embedding_dim, name="embedding"))
modelsequ1.add(layers.GlobalAveragePooling1D())
modelsequ1.add(layers.Dense(100, activation='relu', name="Hidden_layer"))
modelsequ1.add(layers.Dense(50, activation='relu', name="Hidden_layer_2"))
modelsequ1.add(layers.Dense(4, activation='softmax', name="Output_layer"))
modelsequ1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', 'mse']) 


In [59]:
batch_size = 100
epochs = 10

# Fit the model using the train and test datasets.
modelsequ1.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/10
97/97 [==============================] - 3s 23ms/step - loss: 0.8693 - acc: 0.4263 - mse: 0.1423 - val_loss: 0.8318 - val_acc: 0.4223 - val_mse: 0.1419
Epoch 2/10
97/97 [==============================] - 2s 18ms/step - loss: 0.8211 - acc: 0.4072 - mse: 0.1438 - val_loss: 1.0520 - val_acc: 0.2066 - val_mse: 0.1891
Epoch 3/10
97/97 [==============================] - 2s 19ms/step - loss: 1.1361 - acc: 0.3348 - mse: 0.1797 - val_loss: 1.4425 - val_acc: 0.4223 - val_mse: 0.2108
Epoch 4/10
97/97 [==============================] - 2s 19ms/step - loss: 4.3957 - acc: 0.3127 - mse: 0.2583 - val_loss: 3.4620 - val_acc: 0.1777 - val_mse: 0.3583
Epoch 5/10
97/97 [==============================] - 2s 20ms/step - loss: 10.4776 - acc: 0.3056 - mse: 0.2874 - val_loss: 19.5853 - val_acc: 0.4223 - val_mse: 0.2405
Epoch 6/10
97/97 [==============================] - 2s 18ms/step - loss: 16.9414 - acc: 0.3049 - mse: 0.2873 - val_loss: 15.8044 - val_acc: 0.2066 - val_mse: 0.3484
Epoch 7/10
97/97 [

In [60]:
modelsequ1.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_6 (TextV  (None, 200)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 200, 100)          20000     
                                                                 
 global_average_pooling1d_6   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 Hidden_layer (Dense)        (None, 100)               10100     
                                                                 
 Hidden_layer_2 (Dense)      (None, 50)                5050      
                                                                 
 Output_layer (Dense)        (None, 4)                

In [61]:
modelsequ1.evaluate(sentences_test,y_test_categorical)

76/76 [==============================] - 0s 5ms/step - loss: 230.6767 - acc: 0.4223 - mse: 0.2405


[230.67665100097656, 0.4222680330276489, 0.24051547050476074]

In [62]:
pred = modelsequ1.predict(sentences_test, batch_size=100, verbose=1) 
predicted = np.argmax(pred, axis=1)  
report = classification_report(np.argmax(y_test_categorical, axis=1), predicted)  
print(report)

25/25 [==============================] - 0s 9ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       469
           1       0.00      0.00      0.00       501
           2       0.00      0.00      0.00       431
           3       0.42      1.00      0.59      1024

    accuracy                           0.42      2425
   macro avg       0.11      0.25      0.15      2425
weighted avg       0.18      0.42      0.25      2425



c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Vectorize Layer count

In [63]:
vectorize_layer_count = TextVectorization(
    ngrams=None, 
    max_tokens=200, vocabulary=None,
    output_mode='count', output_sequence_length=None, pad_to_max_tokens=True,
)

vectorize_layer_count.adapt(text_dataset)

In [64]:
embedding_dim = 100
vocab_size = len(vectorize_layer_count.get_vocabulary())
vocab_size

200

In [65]:
modelsequ2 = Sequential()
modelsequ2.add(vectorize_layer_count)
modelsequ2.add(layers.Embedding(vocab_size, embedding_dim, name="embedding"))
modelsequ2.add(layers.GlobalAveragePooling1D())
modelsequ2.add(layers.Dense(100, activation='relu', name="Hidden_layer"))
modelsequ2.add(layers.Dense(50, activation='relu', name="Hidden_layer_2"))
modelsequ2.add(layers.Dense(4, activation='softmax', name="Output_layer"))
modelsequ2.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc', 'mse']) 


In [66]:
# Fit the model using the train and test datasets.
batch_size = 100
epochs = 10

modelsequ2.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/10
97/97 [==============================] - 4s 30ms/step - loss: 1.0422 - acc: 0.4225 - mse: 0.1546 - val_loss: 1.0183 - val_acc: 0.4223 - val_mse: 0.1512
Epoch 2/10
97/97 [==============================] - 3s 27ms/step - loss: 0.9621 - acc: 0.4301 - mse: 0.1454 - val_loss: 0.9575 - val_acc: 0.4223 - val_mse: 0.1449
Epoch 3/10
97/97 [==============================] - 2s 25ms/step - loss: 0.9124 - acc: 0.4301 - mse: 0.1408 - val_loss: 0.9215 - val_acc: 0.4223 - val_mse: 0.1423
Epoch 4/10
97/97 [==============================] - 3s 27ms/step - loss: 0.8825 - acc: 0.4301 - mse: 0.1389 - val_loss: 0.8984 - val_acc: 0.4223 - val_mse: 0.1414
Epoch 5/10
97/97 [==============================] - 2s 25ms/step - loss: 0.8626 - acc: 0.4301 - mse: 0.1383 - val_loss: 0.8824 - val_acc: 0.4223 - val_mse: 0.1411
Epoch 6/10
97/97 [==============================] - 3s 27ms/step - loss: 0.8487 - acc: 0.4301 - mse: 0.1381 - val_loss: 0.8704 - val_acc: 0.4223 - val_mse: 0.1410
Epoch 7/10
97/97 [====

In [67]:
modelsequ2.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_7 (TextV  (None, 200)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 200, 100)          20000     
                                                                 
 global_average_pooling1d_7   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 Hidden_layer (Dense)        (None, 100)               10100     
                                                                 
 Hidden_layer_2 (Dense)      (None, 50)                5050      
                                                                 
 Output_layer (Dense)        (None, 4)                

In [68]:
modelsequ2.evaluate(sentences_test,y_test_categorical)

76/76 [==============================] - 0s 5ms/step - loss: 0.8867 - acc: 0.4223 - mse: 0.1439


[0.8866713047027588, 0.4222680330276489, 0.1439012885093689]

In [69]:
pred = modelsequ2.predict(sentences_test, batch_size=100, verbose=1) 
predicted = np.argmax(pred, axis=1)  
report = classification_report(np.argmax(y_test_categorical, axis=1), predicted)  
print(report)

25/25 [==============================] - 0s 10ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       469
           1       0.00      0.00      0.00       501
           2       0.00      0.00      0.00       431
           3       0.42      1.00      0.59      1024

    accuracy                           0.42      2425
   macro avg       0.11      0.25      0.15      2425
weighted avg       0.18      0.42      0.25      2425



c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Vectorize layer int

In [70]:
vectorize_layer_int = TextVectorization(
    ngrams=None, 
    max_tokens=200, vocabulary=None,
    output_mode='int', output_sequence_length=None,
)

vectorize_layer_int.adapt(text_dataset)

In [71]:
embedding_dim = 100
vocab_size = len(vectorize_layer_int.get_vocabulary())
vocab_size

200

In [72]:
modelsequ3 = Sequential()
modelsequ3.add(vectorize_layer_int)
modelsequ3.add(layers.Embedding(vocab_size, embedding_dim, name="embedding"))
modelsequ3.add(layers.GlobalAveragePooling1D())
modelsequ3.add(layers.Dense(100, activation='relu', name="Hidden_layer"))
modelsequ3.add(layers.Dense(50, activation='relu', name="Hidden_layer_2"))
modelsequ3.add(layers.Dense(4, activation='softmax', name="Output_layer"))
modelsequ3.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc', 'mse']) 

In [73]:
# Fit the model using the train and test datasets.
modelsequ3.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/10
97/97 [==============================] - 3s 20ms/step - loss: 1.0171 - acc: 0.4279 - mse: 0.1516 - val_loss: 0.9839 - val_acc: 0.4223 - val_mse: 0.1474
Epoch 2/10
97/97 [==============================] - 2s 18ms/step - loss: 0.9286 - acc: 0.4301 - mse: 0.1420 - val_loss: 0.9323 - val_acc: 0.4223 - val_mse: 0.1430
Epoch 3/10
97/97 [==============================] - 2s 16ms/step - loss: 0.8909 - acc: 0.4301 - mse: 0.1394 - val_loss: 0.9078 - val_acc: 0.4223 - val_mse: 0.1420
Epoch 4/10
97/97 [==============================] - 2s 19ms/step - loss: 0.8709 - acc: 0.4301 - mse: 0.1387 - val_loss: 0.8923 - val_acc: 0.4223 - val_mse: 0.1417
Epoch 5/10
97/97 [==============================] - 2s 16ms/step - loss: 0.8575 - acc: 0.4301 - mse: 0.1384 - val_loss: 0.8804 - val_acc: 0.4223 - val_mse: 0.1415
Epoch 6/10
97/97 [==============================] - 2s 17ms/step - loss: 0.8469 - acc: 0.4301 - mse: 0.1383 - val_loss: 0.8704 - val_acc: 0.4223 - val_mse: 0.1413
Epoch 7/10
97/97 [====

In [74]:

modelsequ3.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_8 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, None, 100)         20000     
                                                                 
 global_average_pooling1d_8   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 Hidden_layer (Dense)        (None, 100)               10100     
                                                                 
 Hidden_layer_2 (Dense)      (None, 50)                5050      
                                                                 
 Output_layer (Dense)        (None, 4)                

In [75]:
modelsequ3.evaluate(sentences_test,y_test_categorical)

76/76 [==============================] - 0s 4ms/step - loss: 0.8835 - acc: 0.4223 - mse: 0.1429


[0.8835266828536987, 0.4222680330276489, 0.1429128348827362]

In [76]:
pred = modelsequ3.predict(sentences_test, batch_size=100, verbose=1) 
predicted = np.argmax(pred, axis=1)  
report = classification_report(np.argmax(y_test_categorical, axis=1), predicted)  
print(report)

25/25 [==============================] - 0s 8ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       469
           1       0.00      0.00      0.00       501
           2       0.00      0.00      0.00       431
           3       0.42      1.00      0.59      1024

    accuracy                           0.42      2425
   macro avg       0.11      0.25      0.15      2425
weighted avg       0.18      0.42      0.25      2425



c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
